In [1]:
!pip install evaluate
!pip install rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ba042e23ed4914313673b03a02fd27ecd412b48256eed0c0aee4676bc07e36be
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
import numpy as np
import pandas as pd
from datasets import Dataset
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import evaluate
rouge = evaluate.load('rouge')
from transformers import BartConfig, BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments

2024-04-02 15:36:50.913947: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 15:36:50.914117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 15:36:51.045571: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


We will be using facebook's barbe-base model (the large cnn model takes too long and uses way too much memory)

In [17]:
configuration = BartConfig()
model_name = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_name).to('cuda:0')
tokenizer = BartTokenizer.from_pretrained(model_name)
tokenizer("Hello world")["input_ids"]

[0, 31414, 232, 2]

In [16]:
df = pd.read_csv("/kaggle/input/cnbc-dataset/cnbc_news_datase.csv").sample(frac=1)
df = df.drop(df[pd.isnull(df['short_description'])].index)
df = df.drop(df[pd.isnull(df['description'])].index)
df = df.reset_index(drop = True)

# We will perform the split here. 90 - 10 split
df_train = df[:int(len(df)*.90)]
df_test  = df[int(len(df)*.90):]

In [5]:
len(df_train), len(df_test)

(521, 58)

In [6]:
df.head(3)

,title,url,published_at,author,publisher,short_description,keywords,header_image,raw_description,description,scraped_at
0,Ping An Not Concerned about Pru-AIG deal,https://www.cnbc.com/2010/03/05/ping-an-not-co...,2010-03-05T08:30:05+0000,NaN,CNBC,"Ping An Insurance, China's No. 2 insurer, said...","cnbc, Articles, Business News, Finance, Banks,...",https://sc.cnbcfm.com/applications/cnbc.com/st...,"<div class=""group""><p><strong>Ping An Insuranc...","Ping An Insurance, China's No. 2 insurer, said...",2021-10-30 14:12:15.245009
1,VMware CEO Pat Gelsinger says a Dell spinoff c...,https://www.cnbc.com/2020/09/29/vmware-ceo-del...,2020-09-29T15:55:01+0000,Jordan Novet,CNBC,VMware CEO Pat Gelsinger said that if majority...,"cnbc, Articles, Enterprise, Breaking News: Tec...",https://image.cnbcfm.com/api/v1/image/10609215...,"<div class=""group""><p><a href=""//www.cnbc.com/...",VMware CEO Pat Gelsinger said that if majority...,2021-10-30 14:11:32.264676
2,Small businesses that pulled outrageous public...,https://www.cnbc.com/2013/05/20/Small-business...,2013-05-20T17:39:16+0000,Daniel Bukszpan,CNBC,Anyone running a small business knows that on...,"cnbc, Articles, Meta Platforms Inc, eBay Inc, ...",https://image.cnbcfm.com/api/v1/image/10075200...,"<div class=""group""><p> Anyone running a <a hr...",Anyone running a small business knows that on...,2021-10-30 14:11:38.559615


In [7]:
def tokenize(example):
    text = example['description']
    
    model_inputs = tokenizer(text, max_length=512, padding='max_length', truncation=True)
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(example['short_description'], max_length=512, padding='max_length', truncation=True)
        model_inputs['label'] = targets['input_ids']
    
    return model_inputs

In [8]:
ds = Dataset.from_dict({
    "article_title": [str(x) for x in df_train["title"]],
    "description": [str(x) for x in df_train["description"]],
    "short_description": [str(x) for x in df_train["short_description"]],
})
ds_train = ds.map(tokenize)

  0%|          | 0/521 [00:00<?, ?ex/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [9]:
ds = Dataset.from_dict({
    "article_title": [str(x) for x in df_test["title"]],
    "description": [str(x) for x in df_test["description"]],
    "short_description": [str(x) for x in df_test["short_description"]],
})
ds_test = ds.map(tokenize)

  0%|          | 0/58 [00:00<?, ?ex/s]

In [22]:
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
rougeLsum_scores = []
for i in tqdm(range(len(ds_test))):
    ids = model.generate(torch.LongTensor([ds_test['input_ids'][i]]).to('cuda:0'), min_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)
    sentence = tokenizer.decode(ids[0], skip_special_tokens=True)
    bleu_scores.append(sentence_bleu(ds_test['short_description'][i].split(), sentence.split()))
    
    rouge_score = rouge.compute(predictions = [sentence], references=[ds_test['short_description'][i]])
    rouge1_scores.append(rouge_score['rouge1'])
    rouge2_scores.append(rouge_score['rouge2'])
    rougeL_scores.append(rouge_score['rougeL'])
    rougeLsum_scores.append(rouge_score['rougeLsum'])

print("BLEU:      ", np.mean(np.array(bleu_scores)))
print("ROUGE1:    ", np.mean(np.array(rouge1_scores)))
print("ROUGE2:    ", np.mean(np.array(rouge2_scores)))
print("ROUGEL:    ", np.mean(np.array(rougeL_scores)))
print("ROUGELSUM: ", np.mean(np.array(rougeLsum_scores)))

100%|██████████| 58/58 [00:29<00:00,  1.94it/s]

BLEU:       0.29698825848898686
ROUGE1:     0.10107524936141932
ROUGE2:     0.007474920515295864
ROUGEL:     0.08953898327987818
ROUGELSUM:  0.08953898327987818


Although a learning rate of 1e-3 is big, it is pretty much required to see any effect. The model is trained on a dataset much bigger than mine and using a tiny learning rate would hardly affect the parameters.

In [23]:
training_args = TrainingArguments(
    ".",
    learning_rate=1e-3,
    per_device_eval_batch_size= 1,
    per_gpu_eval_batch_size= 1,
    weight_decay=0.01,
    num_train_epochs = 1,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train.remove_columns(['article_title', 'description', 'short_description']),
    tokenizer=tokenizer,
    compute_metrics=rouge
)

In [24]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=66, training_loss=5.793571240974195, metrics={'train_runtime': 43.8086, 'train_samples_per_second': 11.893, 'train_steps_per_second': 1.507, 'total_flos': 158836347371520.0, 'train_loss': 5.793571240974195, 'epoch': 1.0})

In [25]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [26]:
bleu_scores2 = []
rouge1_scores2 = []
rouge2_scores2 = []
rougeL_scores2 = []
rougeLsum_scores2 = []
for i in tqdm(range(len(ds_test))):
    ids = model.generate(torch.LongTensor([ds_test['input_ids'][i]]).to('cuda:0'), min_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)
    sentence = tokenizer.decode(ids[0], skip_special_tokens=True)
    bleu_scores2.append(sentence_bleu(ds_test['short_description'][i].split(), sentence.split()))
    
    rouge_score = rouge.compute(predictions = [sentence], references=[ds_test['short_description'][i]])
    rouge1_scores2.append(rouge_score['rouge1'])
    rouge2_scores2.append(rouge_score['rouge2'])
    rougeL_scores2.append(rouge_score['rougeL'])
    rougeLsum_scores2.append(rouge_score['rougeLsum'])

print("BLEU:      ", str(np.mean(np.array(bleu_scores)))[:10], "->", str(np.mean(np.array(bleu_scores2)))[:10])
print("ROUGE1:    ", str(np.mean(np.array(rouge1_scores)))[:10], "->", str(np.mean(np.array(rouge1_scores2)))[:10])
print("ROUGE2:    ", str(np.mean(np.array(rouge2_scores)))[:10], "->", str(np.mean(np.array(rouge2_scores2)))[:10])
print("ROUGEL:    ", str(np.mean(np.array(rougeL_scores)))[:10], "->", str(np.mean(np.array(rougeL_scores2)))[:10])
print("ROUGELSUM: ", str(np.mean(np.array(rougeLsum_scores)))[:10], "->", str(np.mean(np.array(rougeLsum_scores2)))[:10])

 16%|█▌        | 9/58 [00:04<00:24,  1.98it/s]/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
100%|██████████| 58/58 [00:29<00:00,  1.97it/s]

BLEU:       0.29698825 -> 0.59769479
ROUGE1:     0.10107524 -> 0.03676260
ROUGE2:     0.00747492 -> 0.00128055
ROUGEL:     0.08953898 -> 0.03473202
ROUGELSUM:  0.08953898 -> 0.03473202


This is unfortunately, the best score I can get. The bleu precision scores get much higher but the recall rouge scores decrease. The words that my model returns may get better but the ordering, the strings used becomes much less accurate.

The impact of my hyperparameters did a lot. For example, lowering my learning rate would cause virtually no change but if I made it higher, there is a drastic impact on my returning scores. BART is an LLM designed to be used for sequence to sequence tasks. That I used it for a single sentence only might have impacted the code alot more than I had presumed as well. 

# **Task 2 Part 1**

Autonomous Vehicle Navigation
Imagine an autonomous vehicle (AV) that needs to navigate through a city. The actions the vehicle can take at each location include moving north, south, east, west, or staying in place. The outcomes of these actions are partly deterministic and partly stochastic due to factors like weather, traffic conditions, and road closures.

States (S): Each road represents a state that the AV can be in. The could also be special states like the vehicle's destination or hazardous locations to avoid.

Actions (A): At each state, the vehicle can choose from a set of actions: move north, south, east, west, or stay in place.

Transition Probabilities (T): These is a chance that taking an action in a state will lead to a particular subsequent state. For example, if the vehicle decides to move north, there couold be a high probability of successfully moving north, but there's also a small chance that it might stay in place due to traffic or move to a different adjacent state due to slippery roads.

Rewards (R): Each action taken by the vehicle results in a reward or penalty. Moving closer to the destination might result in a positive reward, while moving into a traffic jam or hazardous location might result in a penalty. The goal is to maximize cumulative rewards, effectively guiding the vehicle to the destination efficiently and safely.

Decision-Making Process:
The autonomous vehicle uses the MDP framework to make decisions at each step. The vehicle chooses the action that maximizes its expected reward, using only its current location and rewards process while taking into account the future states it might end up in. The vehicle continuously updates its decisions as it navigates through the city, adapting to new situations (like sudden road closures or changes in traffic conditions) by reassessing its options at each state according to the MDP model.

# **Task 2 Part 2**

Reinforcement learning has always been a substandard alternative to its counterpart, neural networks. However, over the recent years, reinforcement learning has been gaining a lot of attention for all of its successes.

One of the many applications for reinforcement learning is autonomous drone control. Paper can be found here: https://web.stanford.edu/class/aa228/reports/2019/final62.pdf

Using a quadcoptor, Stanford's research team has made it so that it is capable of picking a target and moving there efficiently. This can help solve problems such as victim searching after natural disasters, when people are unable to go into a dangerous place because of the risk - drones would be much more averse to these problems. Perhaps, if the environment were a fallen building and there are survivors trapped inside, the drone would reward itself if it gets closer to signs of survivors such as cries for help or their general existence. The environmental states would correspond to its own environment as well. For example, walls with risks of falling or crumbling would return higher probabilities of being  avoided to the drone. 

The stanford paper I have linked above only shows a rudimentary version of this problem: a drone that picks out a target and flies over to it. However, in the future, the applications for this could be enormous.